In [ ]:
import json
import os
import pandas as pd

In [ ]:
output_folder = "./output_chapters/"
chapters_file_name = "chapters.json"

In [ ]:
chapters_file_path = output_folder + chapters_file_name

# Confirm that file exists
if not os.path.exists(chapters_file_path):
    print(f"Error: The file {chapters_file_path} does not exist.")
    exit(1)
else:
    print(f"File {chapters_file_path} exists.")


In [ ]:
chapters = {}
# read the json and save it as dictionary
with open(chapters_file_path, 'r') as file:
    try:
        chapters = json.load(file)
        print("JSON file loaded successfully.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        exit(1)

In [ ]:
print(f"Number of chapters: {len(chapters)}")
# Get the chapter called "Introduction"
introduction_chapter = chapters["introduction"]

print(f"Introduction chapter:\n{introduction_chapter.get('content', '')[:500]}")

### That is ugly
As you can see, we have data...but it is raw. We now need to divide it, but how do we do so?
### Things to consider:
- What is a line?  
- How do we define where it starts and ends?  
- What happens with sentences that -  
finish in different lines?
- Or sentences that start in one

page and then finish in another one?
- What punctuation do we use?  
- _Why is a sentence?_  

In [ ]:
raw_chapter_text = introduction_chapter.get("content", "")
print(f"Raw chapter text length: {len(raw_chapter_text)} characters")

In [ ]:
# Function to split text into chunks of n lines using line breaks
def split_text_by_lines(text, lines_per_chunk=5):
    lines = text.splitlines()
    chunks = []
    for i in range(0, len(lines), lines_per_chunk):
        chunk = "\n".join(lines[i:i+lines_per_chunk])
        chunks.append({"chunk_id": i // lines_per_chunk + 1, "text": chunk})
    return chunks

In [ ]:
chunks_by_line_jump = split_text_by_lines(raw_chapter_text, lines_per_chunk=1) # Using 1 line per chunk for clarity
df_line_jump = pd.DataFrame(chunks_by_line_jump)
df_line_jump.to_csv(output_folder + "lines_chunk.csv", index=False)
print(df_line_jump.head(10))  # Display the first 10 chunks

In [ ]:
# Function to split text into chunks of n sentences, saving extra info for each chunk
def split_text_by_sentences(text, sentences_per_chunk=5):
    import re
    # Replace line breaks with spaces to avoid breaking sentences
    clean_text = re.sub(r'\s*\n\s*', ' ', text)
    # Split by period, question mark, or exclamation mark followed by space or end of string
    sentences = re.split(r'(?<=[.!?])\s+', clean_text)
    # Remove empty sentences
    sentences = [s.strip() for s in sentences if s.strip()]
    chunks = {}
    for idx, i in enumerate(range(0, len(sentences), sentences_per_chunk)):
        chunk_sentences = sentences[i:i+sentences_per_chunk]
        chunk_text = " ".join(chunk_sentences)
        key = f"{i}-{i+len(chunk_sentences)-1}"
        chunks[key] = {
            "chunk_id": idx,
            "range": key,
            "text": chunk_text,
            "num_characters": len(chunk_text)
        }
    return clean_text, chunks

In [ ]:
# Split into sentence chunks with sentence range as key
clean_text, chunks_by_sentence = split_text_by_sentences(text=raw_chapter_text, sentences_per_chunk=1)  # or >1 for multi-sentence chunks

# Save as JSON: key = sentence range, value = chunk text
output_json_path = os.path.join(output_folder, "sentences_chunks.json")
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(chunks_by_sentence, f, ensure_ascii=False, indent=2)

print(f"Sentence chunks JSON saved to {output_json_path}")

It is still not perfect. For example, if you mention "A. Bonavides," it might be incorrectly split into two sentences. Or, you may want to save more information, such as grouping text by chapter (e.g., "Ability Score Increase"), which requires more advanced, semantic division of the PDF.  
But to get started, this approach is more than enough!

In [ ]:
# Function to split text into chunks of n sentences, avoiding splits at abbreviations/initials
def split_text_by_sentences_reg(text, sentences_per_chunk=5):
    import re

    # List of common abbreviations and initials to protect
    abbreviations = [
        r'(?:[A-Z]\.){2,}',         # e.g., "U.S.", "A.B."
        r'(?:Mr|Mrs|Ms|Dr|Prof|Sr|Jr|St|vs|etc|e\.g|i\.e)\.',  # common abbreviations
    ]
    # Protect abbreviations by replacing the period with a placeholder
    protected_text = text
    for abbr in abbreviations:
        protected_text = re.sub(abbr, lambda m: m.group(0).replace('.', '<DOT>'), protected_text)

    # Replace line breaks with spaces to avoid breaking sentences
    clean_text = re.sub(r'\s*\n\s*', ' ', protected_text)
    # Split by period, question mark, or exclamation mark followed by space or end of string
    sentences = re.split(r'(?<=[.!?])\s+', clean_text)
    # Restore protected periods
    sentences = [s.replace('<DOT>', '.') for s in sentences]
    # Remove empty sentences
    sentences = [s.strip() for s in sentences if s.strip()]
    chunks = []
    for i in range(0, len(sentences), sentences_per_chunk):
        chunk = " ".join(sentences[i:i+sentences_per_chunk])
        chunks.append({"chunk_id": i // sentences_per_chunk + 1, "text": chunk})
    return chunks

In [ ]:
chunks_by_sentence_reg = split_text_by_sentences_reg(text=raw_chapter_text, sentences_per_chunk=1) # Using 1 sentence per chunk for clarity
df_sentences_reg = pd.DataFrame(chunks_by_sentence_reg)

# Show all rows and columns, and prevent text truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

print(df_sentences_reg.head(10))  # Display the first 10 chunks by sentences
df_sentences_reg.to_csv(output_folder + "sentences_chunks.csv", index=False)

# Return settings to default
pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

### Now actually get a chapter to use


In [ ]:
chapter = chapters["2"]
chapter_raw_text = chapter.get("content", "")
chapter_title = chapter.get("title", "No Title")
print(f"{chapter_title} content:\n{chapter_raw_text[:500]}")

In [ ]:
sentences_in_chunk = 10
clean_text, chunks_by_sentence = split_text_by_sentences(text=chapter_raw_text, sentences_per_chunk=sentences_in_chunk)


chapter2_json = {
    "time_created": pd.Timestamp.now().isoformat(),
    "title": chapter_title,
    "sentences_in_chunk": sentences_in_chunk,
    "total_characters": len(clean_text),
    "total_chunks": len(chunks_by_sentence),
    "content": chunks_by_sentence
}

output_json_path = os.path.join(output_folder, "chapter2_chunks_with_info.json")
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(chapter2_json, f, ensure_ascii=False, indent=2)

print(f"Chapter 2 chunked JSON with info saved to {output_json_path}")

In [ ]:
def verify_total_characters(json_path):
    """
    Verifies that the sum of all chunk character counts matches the 'total_characters' field in the JSON,
    allowing up to 1% difference.
    Prints the result and returns True if within tolerance, False otherwise.
    """

    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    total_characters = data.get("total_characters")
    content = data.get("content", {})

    # Sum the num_characters field for each chunk
    sum_characters = sum(chunk["num_characters"] for chunk in content.values())

    print(f"total_characters field: {total_characters}")
    print(f"Sum of all chunk characters: {sum_characters}")

    if total_characters == 0:
        print("❌ total_characters is zero, cannot compare.")
        return False

    percent_diff = abs(total_characters - sum_characters) / total_characters

    print(f"Percent difference: {percent_diff:.4%}")

    if percent_diff <= 0.01:
        print("✅ The total is within 1% of the sum of all chunks.")
        return True
    else:
        print("❌ The total differs by more than 1% from the sum of all chunks.")
        return False

verify_total_characters(os.path.join(output_folder, "chapter2_chunks_with_info.json"))

In [ ]:
def additional_chunk_info(json_path):
    """
    Reads the JSON file and calculates additional statistics about the chunks.
    Prints the min, max, and average number of characters per chunk.
    Also prints which chunk number has the min and max.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    content = data.get("content", {})
    num_characters = [chunk["num_characters"] for chunk in content.values()]
    chunk_keys = list(content.keys())

    if not num_characters:
        print("No chunks found.")
        return

    min_chars = min(num_characters)
    max_chars = max(num_characters)
    avg_chars = sum(num_characters) / len(num_characters)

    min_idx = num_characters.index(min_chars)
    max_idx = num_characters.index(max_chars)

    print(f"Minimum characters in a chunk: {min_chars} (chunk: {chunk_keys[min_idx]})")
    print(f"Maximum characters in a chunk: {max_chars} (chunk: {chunk_keys[max_idx]})")
    print(f"Average characters per chunk: {avg_chars:.2f}")

additional_chunk_info(os.path.join(output_folder, "chapter2_chunks_with_info.json"))
# This is fine cause 1000 characters are around 250 tokens, which is a good size for LLMs even if we send multiple chunks

This is a great example on how adding the regex for the knwon acronyms is great :)
```
"550-559": {
    "chunk_id": 55,  
    "range": "550-559",  
    "text": "line (Dex. save) Brass Fire 5 by 30 ft. line (Dex. save) Bronze Lightning 5 by 30 ft. line (Dex. save) Copper Acid 5 by 30 ft. line (Dex. save) Cold Fire 15 ft. cone (Dex. save) Green Poison 15 ft.",  
    "num_characters": 197  
},
```